In [5]:
import pandas as pd
import pickle
from statsmodels.tsa.arima.model import ARIMA

# Load data
df = pd.read_csv("db_growth_data.csv", parse_dates=["Date"])
df = df.sort_values(["Server", "Database", "Date"])

models = {}

for (server, db), group in df.groupby(["Server", "Database"]):
    # Monthly aggregation (last recorded value in the month)
    monthly_data = group.resample("MS", on="Date")["DB_Size_GB"].last()
    
    # Fill missing months
    ts = monthly_data.asfreq("MS").fillna(method="ffill")
    
    # Fit ARIMA
    model = ARIMA(ts, order=(1, 1, 1))
    model_fit = model.fit()
    
    # Save model in dictionary
    models[(server, db)] = model_fit

# Save all models into a pickle file
with open("arima_models.pkl", "wb") as f:
    pickle.dump(models, f)

print("✅ Models saved successfully!")


C:\Users\vpotla\AppData\Local\Temp\ipykernel_7716\3537569424.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ts = monthly_data.asfreq("MS").fillna(method="ffill")
C:\Users\vpotla\AppData\Local\Temp\ipykernel_7716\3537569424.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ts = monthly_data.asfreq("MS").fillna(method="ffill")
C:\Users\vpotla\Desktop\pvani\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\vpotla\AppData\Local\Temp\ipykernel_7716\3537569424.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ts = monthly_data.asfreq("MS").fillna(method

✅ Models saved successfully!
